# Processos de ETL

Para realizar os processos de extração e transformação, usaremos a linguagem Python. Depois disso, as tabelas serão salvas em arquivos .csv, para depois serem diretamente carregadas no Power BI.

In [1]:
import psycopg2
import pandas as pd


/Users/tayronemonteiro/Desktop/python_projects/heloo_bi/.venv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Primeiro, vamos nos conectar ao banco de dados.

In [2]:
DB_NAME = "heloo_bi"
DB_USER = "postgres"
DB_PASS = "tayrone250595"
DB_HOST = "localhost"

conn = psycopg2.connect(dbname = DB_NAME, user = DB_USER, password = DB_PASS, host = DB_HOST)

cursor = conn.cursor()

Tabela "médicos":

In [3]:
cursor.execute('SELECT * FROM medicos')
medicos = pd.DataFrame(cursor.fetchall(), columns = ['id', 'crm', 'nome', 'especialidade', 'telefone'])

Como um exemplo de transformação para a tabela "médicos", podemos remover as porções "CRM" e "/PB" dos valores da variável "crm", e transformá-la em uma variável numérica, onde o CRM é representado apenas com um número de quatro algarismos.

In [4]:
medicos['crm'] = medicos['crm'].str.replace('[^0-9]', '')

# medicos.to_csv('./tabelas_etl/medicos.csv')

/var/folders/r4/ffp5q_r11jg085l4scxy233r0000gn/T/ipykernel_7881/2901448730.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  medicos['crm'] = medicos['crm'].str.replace('[^0-9]', '')


Tabela "pacientes":

In [5]:
cursor.execute('SELECT * FROM pacientes')
pacientes = pd.DataFrame(cursor.fetchall(), columns = ['id', 'nome', 'telefone'])

# pacientes.to_csv('./tabelas_etl/pacientes.csv')

Tabela "consultas":

In [6]:
cursor.execute('SELECT * FROM consultas')
consultas = pd.DataFrame(cursor.fetchall(), 
                         columns = ['id', 'paciente_id', 'medico_id', 'data_agendamento', 'situacao_id', 'cidade_id'])

# consultas.to_csv('./tabelas_etl/consultas.csv')

Tabela "cidades":

In [7]:
cursor.execute('SELECT * FROM cidades')
cidades = pd.DataFrame(cursor.fetchall(), columns = ['id', 'nome', 'estado_sigla'])

# cidades.to_csv('./tabelas_etl/cidades.csv')

Tabela "consultas_situações":

In [8]:
cursor.execute('SELECT * FROM consultas_situacoes')
consultas_situacoes = pd.DataFrame(cursor.fetchall(), columns = ['id', 'nome'])

Por fim, fechamos a conexão com a base de dados da qual extraímos os dados.

In [9]:
cursor.close()

conn.close()

del cursor
del conn

Agora, fazemos a conexão com a base que usaremos para carregar os dados.

In [10]:
DB_NAME = "heloo_bi_load"
DB_USER = "postgres"
DB_PASS = "tayrone250595"
DB_HOST = "localhost"

conn = psycopg2.connect(dbname = DB_NAME, user = DB_USER, password = DB_PASS, host = DB_HOST)

cursor = conn.cursor()

Criamos as tabelas também nessa base.

In [11]:
cursor.execute('''CREATE TABLE medicos(
    id BIGSERIAL NOT NULL PRIMARY KEY,
    crm TEXT,
    nome TEXT,
    especialidade TEXT,
    telefone TEXT
    );''')

cursor.execute('''CREATE TABLE pacientes(
    id BIGSERIAL NOT NULL PRIMARY KEY,
    nome TEXT,
    telefone TEXT
    );''')

cursor.execute('''CREATE TABLE consultas_situacoes(
    id BIGSERIAL NOT NULL PRIMARY KEY,
    nome TEXT NOT NULL
    );''')

cursor.execute('''CREATE TABLE cidades(
    id BIGSERIAL NOT NULL PRIMARY KEY,
    nome TEXT NOT NULL,
    estado_sigla TEXT NOT NULL
   );''')

cursor.execute('''CREATE TABLE consultas(
    id BIGSERIAL NOT NULL PRIMARY KEY,
    paciente_id BIGINT NOT NULL REFERENCES pacientes(id), 
    medico_id BIGINT NOT NULL REFERENCES medicos(id), 
    data_agendamento TIMESTAMP,
    situacao_id BIGINT NOT NULL REFERENCES consultas_situacoes(id), 
    cidade_id BIGINT NOT NULL REFERENCES cidades(id)
    );''')

conn.commit()


E agora carregamos os dados na nova base recém criada. Primeiro, a tabela "medicos":

In [12]:
medicos = list(medicos.itertuples(index = False, name = None))

for instancia_medico in medicos:
    cursor.execute('INSERT INTO medicos (id, crm, nome, especialidade, telefone) VALUES (%s, %s, %s, %s, %s)', 
                   instancia_medico)

conn.commit()

Agora, "pacientes":

In [13]:
pacientes = list(pacientes.itertuples(index = False, name = None))

for instancia_paciente in pacientes:
    cursor.execute('INSERT INTO pacientes (id, nome, telefone) VALUES (%s, %s, %s)', 
                   instancia_paciente)

conn.commit()

Tabela "consultas_situacoes":

In [14]:
consultas_situacoes = list(consultas_situacoes.itertuples(index = False, name = None))

for instancia_situacao in consultas_situacoes:
    cursor.execute('''INSERT INTO consultas_situacoes (id, nome) VALUES (%s, %s)''', instancia_situacao)

conn.commit()

Tabela "cidades":

In [15]:
cidades = list(cidades.itertuples(index = False, name = None))

for instancia_cidade in cidades:
    cursor.execute('INSERT INTO cidades (id, nome, estado_sigla) VALUES (%s, %s, %s)', instancia_cidade)

conn.commit()

Tabela "consultas":

In [16]:
consultas = list(consultas.itertuples(index = False, name = None))

for instancia_consulta in consultas:
    cursor.execute('''INSERT INTO consultas (id, paciente_id, medico_id, data_agendamento, 
                   situacao_id, cidade_id) VALUES (%s, %s, %s, %s, %s, %s)''', instancia_consulta)

conn.commit()